# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/1. IRONHACK/BBDD/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [6]:
data.shape

(68584, 7)

In [7]:
len(data['ProductName'].unique())

452

In [8]:
# Check the columns that have missing values
data.isnull().sum()

CustomerID     0
FirstName      0
LastName       0
SalesID        0
ProductID      0
ProductName    0
Quantity       0
dtype: int64

In [9]:
data['Quantity'].sum()

885499

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [10]:
df = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
df

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

In [11]:
#row = df.iloc[63627]
#row

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
table = pd.pivot_table(df, values='Quantity', index='ProductName', columns='CustomerID', aggfunc=np.sum, fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,25,25,0,0,0,0,25,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,25,25,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,25,0,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,25,0,0,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,24,0,0,0,0,25,0,0,0,0,25,0,0,25,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,...,24,0,0,0,0,0,0,0,25,0,0,25,0,25,25,25,25,0,0,0,0,0,0,0,0,0,0,0,0,25,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,...,0,0,0,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,25,0,25,0,0,0,0,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25,25,0,0,0,25,0,0,0,0,0,0,0,0,0,0,25,25,25,0,0,0,0,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,25,0,0,25,25,0,0,25,0,0,0,0,0,0


In [13]:
p_table = pd.DataFrame(table, index=list(table.index), columns =list(table.columns))
p_table

,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,25,25,0,0,0,0,25,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,25,25,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,25,0,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,25,0,0,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,24,0,0,0,0,25,0,0,0,0,25,0,0,25,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,...,24,0,0,0,0,0,0,0,25,0,0,25,0,25,25,25,25,0,0,0,0,0,0,0,0,0,0,0,0,25,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,...,0,0,0,25,25,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,25,0,25,0,0,0,0,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25,25,0,0,0,25,0,0,0,0,0,0,0,0,0,0,25,25,25,0,0,0,0,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,25,0,0,25,25,0,0,25,0,0,0,0,0,0
Yogurt - French Vanilla,1,0,0,1,0,0,2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,25,0,25,0,0,0,0,0,25,0,25,0,25,0,0,25,25,0,0,0,25,0,0,0,0,0,0,25


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:
similarity_matrix = squareform(pdist(p_table.T, 'euclidean'))
similarity_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

These distances are smaller for users that are more similar, but for our purposes it would be preferable to have a higher score for users that are more similar and a lower score for users that are less similar. We can easily achieve this by adding 1 to the scores and inverting them. Let's also import them into a data frame so that we can more easily view and analyze them.

In [15]:
distances = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))), index=p_table.columns, columns=p_table.columns)
distances

,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,0.078977,0.083651,0.078977,0.076678,0.083972,0.080920,0.079792,0.085297,0.081800,0.075956,0.076923,0.083020,0.084959,0.078187,0.082100,0.076678,0.081503,0.082100,0.077674,0.085983,0.081800,0.077171,0.076923,0.083333,0.077929,0.075956,0.082403,0.078448,0.077421,0.080350,...,0.004440,0.005273,0.004894,0.004704,0.004348,0.004077,0.005222,0.004529,0.004493,0.004322,0.004431,0.004687,0.004476,0.004551,0.004248,0.004450,0.004948,0.004205,0.004215,0.004531,0.004943,0.004435,0.004802,0.004655,0.004471,0.004975,0.004779,0.004478,0.004032,0.004782,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,0.075025,0.075956,0.076435,0.073478,0.080350,0.074798,0.077171,0.083972,0.074573,0.072844,0.073265,0.076923,0.076923,0.075720,0.077674,0.075720,0.082100,0.076194,0.075720,0.080350,0.076435,0.071824,0.075025,0.079246,0.074573,0.073265,0.077421,0.075486,0.074129,0.078711,...,0.004456,0.005293,0.004915,0.004708,0.004363,0.004083,0.005210,0.004542,0.004497,0.004325,0.004429,0.004691,0.004475,0.004564,0.004240,0.004436,0.004974,0.004227,0.004231,0.004537,0.004918,0.004445,0.004812,0.004641,0.004473,0.004953,0.004775,0.004491,0.004041,0.004783,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,0.080070,0.082403,0.083651,0.077171,0.088152,0.081503,0.079792,0.087410,0.077929,0.076923,0.075956,0.082403,0.081210,0.081503,0.079792,0.076678,0.082709,0.079792,0.076194,0.082709,0.080070,0.076678,0.078977,0.088152,0.074129,0.078448,0.081210,0.081210,0.080070,0.080920,...,0.004448,0.005276,0.004913,0.004699,0.004348,0.004082,0.005218,0.004533,0.004493,0.004312,0.004410,0.004677,0.004467,0.004553,0.004236,0.004439,0.004972,0.004211,0.004209,0.004517,0.004916,0.004444,0.004816,0.004642,0.004474,0.004963,0.004790,0.004467,0.004045,0.004768,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,0.076923,0.076435,0.082403,0.072637,0.085297,0.077171,0.075720,0.082709,0.078448,0.075025,0.075025,0.081210,0.077421,0.074798,0.078187,0.078711,0.079792,0.079246,0.077171,0.083333,0.080634,0.073054,0.075956,0.083333,0.076923,0.074573,0.079518,0.077929,0.075956,0.078187,...,0.004435,0.005293,0.004904,0.004696,0.004345,0.004087,0.005203,0.004526,0.004490,0.004310,0.004427,0.004679,0.004473,0.004558,0.004234,0.004441,0.004956,0.004214,0.004231,0.004524,0.004913,0.004441,0.004807,0.004644,0.004473,0.004963,0.004776,0.004491,0.004042,0.004778,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,0.075956,0.078448,0.081210,0.076194,0.083333,0.074798,0.075255,0.082709,0.078977,0.074573,0.075025,0.081800,0.076923,0.076194,0.077674,0.076678,0.076678,0.078187,0.077171,0.080350,0.078977,0.073910,0.075486,0.079792,0.073693,0.071626,0.076923,0.074573,0.074129,0.078187,...,0.004454,0.005308,0.004918,0.004706,0.004345,0.004087,0.005206,0.004549,0.004499,0.004307,0.004431,0.004676,0.004471,0.004577,0.004230,0.004456,0.004978,0.004220,0.004222,0.004523,0.004918,0.004454,0.004804,0.004652,0.004471,0.004968,0.004784,0.004478,0.004035,0.004789,0.004810,0.004702,0.0

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
#saco una muestra de mi df origianl para obtener random customerID
df_sample = distances.sample()
df_sample

,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
59017,0.007865,0.007839,0.007836,0.007849,0.007877,0.007872,0.007823,0.007852,0.007821,0.00779,0.007811,0.007847,0.007851,0.007838,0.00786,0.007832,0.007848,0.007819,0.007856,0.007825,0.007844,0.007828,0.007841,0.007853,0.007878,0.00786,0.007836,0.007811,0.007847,0.007818,0.007835,0.007837,0.007811,0.007866,0.007822,0.007881,0.007857,0.007841,0.007824,0.007835,...,0.004005,0.004563,0.004282,0.004155,0.003846,0.003822,0.004517,0.004128,0.004272,0.00401,0.004042,0.004332,0.004164,0.004097,0.003839,0.004282,0.004322,0.003931,0.004026,0.004182,0.004327,0.004168,0.004168,0.004337,0.00411,0.004292,0.004287,0.004034,0.003787,0.004229,0.004342,0.004159,0.004119,0.004373,0.003986,0.004182,0.004201,0.003978,0.003894,0.003982


In [17]:
#Paso mi customerID que es mi index a una lista, y luego a valor numérico
value = df_sample.index.to_list()
value
#type(value)
n = value[0]
type(n)

int

In [23]:
my_list = []
my_list.append(n)
#customer_n = p_table.T.filter(my_list, axis=0)
#customer_n = customer_n.T
#my_list = my_list.remove(user_id)
my_list

[59017]

In [24]:
my_list = my_list.remove(n)
print(my_list)

None


In [25]:
#uso mi valor generado para obtener los 5 similares
top_5_similar = distances[n].sort_values(ascending=False)[1:].nlargest(5)

print(f'for the customerID: {n}, the 5 most similiar CustomerIDs are:')
display(top_5_similar)

for the customerID: 59017, the 5 most similiar CustomerIDs are:


20331    0.008057
4644     0.008003
14012    0.007981
5230     0.007965
8962     0.007950
Name: 59017, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [26]:
#obtengo el listado de CustomerIDs similares y lo paso a una lista
customerID_similares = top_5_similar.index.to_list()
customerID_similares

[20331, 4644, 14012, 5230, 8962]

In [27]:
df_2 = p_table.T.filter(customerID_similares, axis=0)
df_2

,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,Bacardi Breezer - Tropical,Bagel - Plain,Baking Powder,Banana - Leaves,Banana Turning,Bananas,Bandage - Fexible 1x3,Bandage - Flexible Neon,Bar - Granola Trail Mix Fruit Nut,"Bar Mix - Pina Colada, 355 Ml",Barramundi,Bay Leaf,Beans - Kidney White,"Beans - Kidney, Canned","Beans - Kidney, Red Dry",Beans - Wax,"Beef - Chuck, Boneless",Beef - Ground Medium,"Beef - Ground, Extra Lean, Fresh",Beef - Inside Round,Beef - Montreal Smoked Brisket,Beef - Prime Rib Aaa,Beef - Rib Eye Aaa,Beef - Short Loin,Beef - Striploin Aa,"Beef - Tenderlion, Center Cut",Beef - Texas Style Burger,Beef - Top Sirloin,Beef - Top Sirloin - Aaa,Beef Ground Medium,...,Whmis - Spray Bottle Trigger,Wiberg Super Cure,Wine - Alsace Gewurztraminer,Wine - Blue Nun Qualitatswein,"Wine - Cahors Ac 2000, Clos",Wine - Chablis 2003 Champs,Wine - Charddonnay Errazuriz,Wine - Chardonnay South,Wine - Crozes Hermitage E.,Wine - Ej Gallo Sierra Valley,Wine - Fume Blanc Fetzer,Wine - Gato Negro Cabernet,Wine - Hardys Bankside Shiraz,Wine - Magnotta - Belpaese,Wine - Magnotta - Cab Sauv,"Wine - Magnotta, Merlot Sr Vqa",Wine - Pinot Noir Latour,Wine - Prosecco Valdobiaddene,"Wine - Red, Colio Cabernet","Wine - Red, Cooking","Wine - Red, Harrow Estates, Cab",Wine - Redchard Merritt,Wine - Ruffino Chianti,Wine - Sogrape Mateus Rose,Wine - Toasted Head,Wine - Two Oceans Cabernet,Wine - Valpolicella Masi,Wine - Vidal Icewine Magnotta,Wine - Vineland Estate Semi - Dry,Wine - White Cab Sauv.on,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
20331,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,6,0,6,0,0,0,0,0,0,0,6,0,6,0,0,0,0,0,0,0,0,6,0,0,...,0,0,0,6,0,6,0,0,0,0,0,0,6,0,0,0,0,0,0,6,0,0,0,0,0,0,6,6,6,0,0,6,0,0,0,0,0,0,0,0
4644,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,2,0,0,0,0,0,2,0,2,2,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
14012,0,0,0,0,0,4,0,4,4,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,4,0,0,0,0,0,0,...,4,0,0,0,0,0,4,4,0,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,8
5230,0,0,0,0,0,4,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,...,2,2,0,0,0,0,2,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2
8962,0,0,0,0,0,0,0,0,0,3,3,0,0,0,3,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,...,6,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [28]:
agg_table = df_2.T
agg_table['Total'] = agg_table.sum(axis=1)
sum_table = agg_table.sort_values('Total', ascending=False)
sum_table

,20331,4644,14012,5230,8962,Total
Cinnamon Buns Sticky,6,0,12,2,0,20
Flour - Whole Wheat,6,0,4,2,3,15
Chicken - Soup Base,12,0,0,0,3,15
Rice - Jasmine Sented,6,0,8,0,0,14
Puree - Mocha,6,0,8,0,0,14
...,...,...,...,...,...,...
"Liners - Banana, Paper",0,0,0,0,0,0
Longos - Chicken Wings,0,0,0,0,0,0
Longos - Grilled Chicken With,0,0,0,0,0,0
Meldea Green Tea Liquor,0,0,0,0,0,0


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [29]:
customer_n = p_table.T.filter(value, axis=0)
customer_n = customer_n.T
customer_n

,59017
Anchovy Paste - 56 G Tube,0
"Appetizer - Mini Egg Roll, Shrimp",0
Appetizer - Mushroom Tart,0
Appetizer - Sausage Rolls,0
Apricots - Dried,0
...,...
Yeast Dry - Fermipan,0
Yoghurt Tubes,0
"Yogurt - Blueberry, 175 Gr",0
Yogurt - French Vanilla,0


In [30]:
concat_df = pd.concat([customer_n, sum_table], axis=1)
concat_df

,59017,20331,4644,14012,5230,8962,Total
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0
Apricots - Dried,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0
"Yogurt - Blueberry, 175 Gr",0,0,0,0,0,0,0
Yogurt - French Vanilla,0,0,0,0,0,0,0


In [31]:
filter = concat_df[concat_df[n] == 0]
filter
filter['Total'].nlargest(5)

Cinnamon Buns Sticky            20
Puree - Mocha                   14
Rice - Jasmine Sented           14
Bacardi Breezer - Tropical      13
Fish - Scallops, Cold Smoked    13
Name: Total, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [32]:
#id_users = data['CustomerID'].to_list()
#unique_id_users = set(id_users)
#unique_id_users = list(unique_id_users)
#len(unique_id_users)
#type(unique_id_users)
#id_users = [33,	200,	264,	356,	412,	464,	477]
#id_users

In [73]:
id_users = data['CustomerID'].to_list()
unique_id_users = set(id_users)
unique_id_users = list(unique_id_users)
#unique_id_users

In [74]:
def customers_recomendations (user_id):
  #step 4: generate a list of the top 5 most similar customers for a specific CustomerID
  distances = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))), index=p_table.columns, columns=p_table.columns)
  top_5_similar = distances[user_id].sort_values(ascending=False)[1:].nlargest(5)
  #obtengo el listado de CustomerIDs similares y lo paso a una lista
  customerID_similares = top_5_similar.index.to_list() 

  #step 5: from the data, select the records for the list of similar CustomerIDs
  df_2 = p_table.T.filter(customerID_similares, axis=0) 

  #step 6: aggregate those customer purchase records by productname, sum the quantify field, and then rank them in descending order by quantity
  agg_table = df_2.T
  agg_table['Total'] = agg_table.sum(axis=1)
  sum_table = agg_table.sort_values('Total', ascending=False)

  #step 7: 
  my_list = []
  my_list.append(user_id)
  customer_n = p_table.T.filter(my_list, axis=0)
  customer_n = customer_n.T
  my_list = my_list.remove(user_id)

  concat_df = pd.concat([customer_n, sum_table], axis=1)

  filter = concat_df[concat_df[user_id] == 0]
  return filter['Total'].nlargest(5)


In [75]:
recomendaciones = {i:customers_recomendations(i) for i in unique_id_users}

In [85]:
#recomendaciones

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [77]:
df_rec1 = pd.DataFrame.from_dict(recomendaciones, orient='index')
df_rec1 = df_rec1.fillna(0)
df_rec1

,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Bagel - Plain,Cup - Translucent 7 Oz Clear,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Bread - Calabrese Baguette,Pop Shoppe Cream Soda,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Cheese - Parmesan Grated,Puree - Mocha,Cheese - Victor Et Berthold,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Beef - Texas Style Burger,Flour - Pastry,Foam Cup 6 Oz,Bacardi Breezer - Tropical,Cheese - Mozzarella,Banana - Leaves,Bread - Hot Dog Buns,Bread - Italian Roll With Herbs,Spinach - Baby,Pork - Hock And Feet Attached,Vinegar - Tarragon,Beans - Wax,Bread Foccacia Whole,Blueberries,Mussels - Cultivated,Turkey - Oven Roast Breast,Vaccum Bag 10x13,Bread Crumbs - Panko,...,Jagermeister,"Beans - Kidney, Red Dry",Cookie Chocolate Chip With,"Liners - Banana, Paper",Wine - Two Oceans Cabernet,Pecan Raisin - Tarts,Peas - Frozen,Muffin Mix - Blueberry,Sauce - Rosee,Pasta - Angel Hair,Wine - Magnotta - Cab Sauv,Steam Pan - Half Size Deep,Smoked Paprika,Wine - Alsace Gewurztraminer,Wine - Pinot Noir Latour,Pail With Metal Handle 16l White,Cheese - Mix,"Wine - White, Mosel Gold","Wine - Red, Cooking",Squid U5 - Thailand,"Turkey - Whole, Fresh","Wine - Red, Colio Cabernet",Garlic - Peeled,"Rum - Coconut, Malibu",Gatorade - Xfactor Berry,"Yogurt - Blueberry, 175 Gr",Wine - Toasted Head,Tomato - Tricolor Cherry,Wine - Vidal Icewine Magnotta,Vanilla Beans,Oil - Safflower,Raspberries - Fresh,"Orange - Canned, Mandarin",Sugar - Fine,Mustard Prepared,Truffle Cups - Brown,"Mushroom - Porcini, Dry",Sprouts - Alfalfa,Remy Red,Soup V8 Roasted Red Pepper
83973,28.0,24.0,22.0,22.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79997,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63742,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96522,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96524,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [78]:
df_rec = pd.DataFrame([[key, recomendaciones[key]] for key in recomendaciones.keys()], columns=['CustomerID', 'Top 5 products'] )
df_rec

,CustomerID,Top 5 products
0,83973,Cocoa Butter 28 Veal - Slab Bacon ...
1,59399,"Shrimp - Baby, Warm Water 15 Knife Plast..."
2,92168,Cup - Translucent 7 Oz Clear 21 Snapple Lem...
3,49159,"Pasta - Penne, Rigate, Dry 12 Cookies - As..."
4,18441,Soupfoamcont12oz 112con 4 Anchovy Paste -...
...,...,...
995,77814,Cheese - Victor Et Berthold 16 Cake - Cake ...
996,32753,Grouper - Fresh 9 Sherry - Dry ...
997,20476,"Pasta - Detalini, White, Fresh 4 Soup - C..."
998,67582,Cake - Mini Cheesecake 17 Ecolab - Solid F...


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [79]:
def customers_recomendations (user_id):
  #step 4: generate a list of the top 5 most similar customers for a specific CustomerID
  distances = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'cityblock'))), index=p_table.columns, columns=p_table.columns)
  top_5_similar = distances[user_id].sort_values(ascending=False)[1:].nlargest(5)
  #obtengo el listado de CustomerIDs similares y lo paso a una lista
  customerID_similares = top_5_similar.index.to_list() 

  #step 5: from the data, select the records for the list of similar CustomerIDs
  df_2 = p_table.T.filter(customerID_similares, axis=0) 

  #step 6: aggregate those customer purchase records by productname, sum the quantify field, and then rank them in descending order by quantity
  agg_table = df_2.T
  agg_table['Total'] = agg_table.sum(axis=1)
  sum_table = agg_table.sort_values('Total', ascending=False)

  #step 7: 
  my_list = []
  my_list.append(user_id)
  customer_n = p_table.T.filter(my_list, axis=0)
  customer_n = customer_n.T
  my_list = my_list.remove(user_id)

  concat_df = pd.concat([customer_n, sum_table], axis=1)

  filter = concat_df[concat_df[user_id] == 0]
  return filter['Total'].nlargest(5)

In [80]:
recomendaciones_cityblock = {i:customers_recomendations(i) for i in unique_id_users}

In [84]:
#recomendaciones_cityblock

In [82]:
df_rec_cityblock = pd.DataFrame.from_dict(recomendaciones_cityblock, orient='index')
df_rec_cityblock = df_rec_cityblock.fillna(0)
df_rec_cityblock

,"Pork - Bacon, Double Smoked",Bar - Granola Trail Mix Fruit Nut,Beef Ground Medium,Butter - Unsalted,Juice - Lime,"Pepper - Black, Whole","Salsify, Organic",Soup - Campbells Bean Medley,Chinese Foods - Chicken,Knife Plastic - White,Muffin Batt - Blueberry Passion,Wine - Blue Nun Qualitatswein,Tia Maria,Veal - Sweetbread,Beef - Montreal Smoked Brisket,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,Cheese - Mozzarella,"Beets - Candy Cane, Organic",Bread - Italian Roll With Herbs,"Sole - Dover, Whole, Fresh",Foam Dinner Plate,Berry Brulee,"Cheese - Boursin, Garlic / Herbs",Chocolate - Dark,Guinea Fowl,Ecolab - Solid Fusion,Sherry - Dry,Ketchup - Tomato,Beans - Wax,Cup - Translucent 7 Oz Clear,"Pasta - Penne, Rigate, Dry",Pork - Inside,Juice - Apple Cider,Pork - Kidney,Squid - Tubes / Tenticles 10/20,"Wine - White, Colubia Cresh",Beer - Blue,Chips Potato Salt Vinegar 43g,...,Otomegusa Dashi Konbu,Sauce - Rosee,Banana - Leaves,Veal - Eye Of Round,Beef - Striploin Aa,"Veal - Brisket, Provimi,bnls","Lemonade - Natural, 591 Ml",Chips Potato All Dressed - 43g,Vol Au Vents,Rabbit - Whole,V8 - Berry Blend,Bread Foccacia Whole,Pastry - Butterscotch Baked,Apricots - Halves,Soup Knorr Chili With Beans,Scallops - Live In Shell,Lambcasing,Dc - Frozen Momji,"Mushroom - Porcini, Dry","Water - Mineral, Natural",Wine - Magnotta - Belpaese,Isomalt,Towels - Paper / Kraft,Longos - Grilled Chicken With,"Mushroom - Trumpet, Dry",Chicken - Wieners,Carbonated Water - Blackcherry,"Cheese - Brie,danish",Puree - Mocha,"Beef - Ground, Extra Lean, Fresh",Assorted Desserts,Muffin - Zero Transfat,Cocoa Butter,Cheese - Victor Et Berthold,Soupcontfoam16oz 116con,Wine - Pinot Noir Latour,Vanilla Beans,"Soup - Campbells, Cream Of",Fuji Apples,"Bread - Roll, Canadian Dinner"
83973,4.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59399,3.0,0.0,0.0,3.0,0.0,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94258,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86167,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4261,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0

In [83]:
df_rec_cityblock2 = pd.DataFrame([[key, recomendaciones_cityblock[key]] for key in recomendaciones_cityblock.keys()], columns=['CustomerID', 'Top 5 products'] )
df_rec_cityblock2

,CustomerID,Top 5 products
0,83973,"Pork - Bacon, Double Smoked 4 Bar - G..."
1,59399,"Pepper - Black, Whole 4 Butter - Uns..."
2,92168,Chinese Foods - Chicken 3 Knife Pla...
3,49159,Butter - Unsalted 4 Tia Maria ...
4,18441,Anchovy Paste - 56 G Tube 3 Bar - G...
...,...,...
995,77814,Bandage - Fexible 1x3 3 Cod - Black...
996,32753,Butter - Unsalted 4 Veal - ...
997,20476,"Pasta - Detalini, White, Fresh 4 Cheese -..."
998,67582,Lettuce - Spring Mix 4 Pork - Kid...


In [ ]:
#Cuando ocupamos cityblocks, el número de productos se reduce